In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs

In [3]:
df_sp = pd.read_csv("../data/Raw/especies en peligro (1).csv")
df_sp.head()

/tmp/ipykernel_2396/3980527193.py:1: DtypeWarning: Columns (11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sp = pd.read_csv("../data/Raw/especies en peligro (1).csv")


,assessment_id,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,legend,subspecies,subpop,dist_comm,island,tax_comm,source,basisofrec,event_year,dec_long,dec_lat
0,497499,132523146,Hubbsina turneri,1,1,1,IUCN Freshwater Biodiversity Unit,2018,"Köck, GWG",Extant (resident),,,,,,de Jong,,2014.0,-101.47950,19.87450
1,497499,132523146,Hubbsina turneri,1,1,1,IUCN Freshwater Biodiversity Unit,2018,"Köck, GWG",Extant (resident),,,,,,Köck et al.,,2014.0,-101.77730,19.82560
2,497499,132523146,Hubbsina turneri,1,1,1,IUCN Freshwater Biodiversity Unit,2018,"Köck, GWG",Extant (resident),,,,,,Lyons,,2015.0,-101.78760,19.82730
3,498476,10769,Ictalurus mexicanus,1,1,1,"Soto-Galera, E. and L. Alcántara-Soria, ENCB-I...",2018,IUCN (International Union for Conservation of ...,Extant (resident),,,,,,,PRESERVED_SPECIMEN,1971.0,-99.35417,21.98083
4,498476,10769,Ictalurus mexicanus,1,1,1,"Soto-Galera, E. and L. Alcántara-Soria, ENCB-I...",2018,IUCN (International Union for Conservation of ...,Extant (resident),,,,,,,PRESERVED_SPECIMEN,1974.0,-99.30000,22.00000


In [4]:
df_sp.shape

(82098, 20)

In [5]:
df_sp.duplicated().sum()

np.int64(0)

In [6]:
# Número de especies

df_sp["basisofrec"].nunique()

15

In [7]:
df_sp = df_sp[[
    "sci_name",
    "presence",
    "origin",
    "seasonal",
    "dec_long",
    "dec_lat"    
]]

df_sp = df_sp.rename(columns = {
    "dec_long": "lon",
    "dec_lat": "lat"
})

df_sp = df_sp.dropna(subset=["lon", "lat"])

# Asegurar coordenadas como float
df_sp["lon"] = df_sp["lon"].astype(float)
df_sp["lat"] = df_sp["lat"].astype(float)

df_sp.head(10)

,sci_name,presence,origin,seasonal,lon,lat
0,Hubbsina turneri,1,1,1,-101.47950,19.87450
1,Hubbsina turneri,1,1,1,-101.77730,19.82560
2,Hubbsina turneri,1,1,1,-101.78760,19.82730
3,Ictalurus mexicanus,1,1,1,-99.35417,21.98083
4,Ictalurus mexicanus,1,1,1,-99.30000,22.00000
5,Ameca splendens,1,1,1,-104.07930,20.90140
6,Ameca splendens,1,1,1,-103.84500,20.68010
7,Ameca splendens,1,1,1,-103.84330,20.68100
8,Ameca splendens,1,1,1,-103.84310,20.67610
9,Ameca splendens,1,1,1,-103.84270,20.67580


In [8]:
dups = df_sp[df_sp.duplicated(subset=["sci_name", "lon", "lat"], keep=False)]
dups.head(10)

,sci_name,presence,origin,seasonal,lon,lat
15,Lucania interioris,1,1,1,-102.099998,26.933332
16,Lucania interioris,1,1,1,-102.107245,26.915160
17,Lucania interioris,1,1,1,-102.107245,26.915160
22,Lucania interioris,1,1,1,-102.099998,26.933332
53,Tlaloc hildebrandi,1,1,1,-92.616670,16.746670
59,Tlaloc hildebrandi,1,1,1,-92.616670,16.746670
61,Tlaloc hildebrandi,1,1,1,-92.581390,16.725280
81,Tlaloc hildebrandi,1,1,1,-92.581390,16.725280
124,Chirostoma promelas,1,1,1,-103.217220,20.289720
125,Chirostoma promelas,1,1,1,-103.182780,20.286390


In [9]:
df_sp = df_sp.drop_duplicates(subset=["sci_name", "lon", "lat"])
df_sp.shape

(70273, 6)

In [10]:
df_sp.head(10)

,sci_name,presence,origin,seasonal,lon,lat
0,Hubbsina turneri,1,1,1,-101.47950,19.87450
1,Hubbsina turneri,1,1,1,-101.77730,19.82560
2,Hubbsina turneri,1,1,1,-101.78760,19.82730
3,Ictalurus mexicanus,1,1,1,-99.35417,21.98083
4,Ictalurus mexicanus,1,1,1,-99.30000,22.00000
5,Ameca splendens,1,1,1,-104.07930,20.90140
6,Ameca splendens,1,1,1,-103.84500,20.68010
7,Ameca splendens,1,1,1,-103.84330,20.68100
8,Ameca splendens,1,1,1,-103.84310,20.67610
9,Ameca splendens,1,1,1,-103.84270,20.67580


In [11]:
df_sp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70273 entries, 0 to 82096
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sci_name  70273 non-null  object 
 1   presence  70273 non-null  int64  
 2   origin    70273 non-null  int64  
 3   seasonal  70273 non-null  int64  
 4   lon       70273 non-null  float64
 5   lat       70273 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 3.8+ MB


In [12]:
df_sp.to_csv("../data/GeoDataFrame/dataframe/df_sp.csv", index = False)

#### GeoDataFrame

In [38]:

df_sp = pd.read_csv("../data/GeoDataFrame/dataframe/df_sp.csv")

In [39]:
gdf_sp = gpd.GeoDataFrame(
    df_sp,
    geometry=gpd.points_from_xy(
        df_sp.lon,
        df_sp.lat
    ),
    crs="EPSG:4326"
)

In [40]:
# coordenadas fuera de rango
gdf_sp = gdf_sp[
    gdf_sp.lon.between(-180, 180) &
    gdf_sp.lat.between(-90, 90)
]

# eliminar geometrías inválidas
gdf_sp = gdf_sp[gdf_sp.is_valid]

In [41]:
gdf_sp.shape

(70273, 8)

In [42]:
gdf_sp_m = gdf_sp.to_crs(epsg=3395)


In [43]:
gdf_sp.to_file("../data/GeoDataFrame/gdf_species.gpkg", driver="GPKG")

In [18]:
assessment = pd.read_csv("../data/GeoDataFrame/assessments.csv")
df_sp = pd.read_csv("../data/GeoDataFrame/dataframe/df_sp.csv")

In [19]:
assessment = assessment.rename(columns={"scientificName":"sci_name"})

In [20]:
assessment_cols = assessment[[
    "sci_name",
    "redlistCategory",        # EN, VU, CR...
    "redlistCriteria"
]]

In [21]:
points_joined = df_sp.merge(
    assessment_cols,
    on="sci_name",
    how="left"
)

In [22]:
points_joined.groupby("redlistCategory").size()

redlistCategory
Critically Endangered     2007
Endangered                6692
Near Threatened          39959
Vulnerable               21615
dtype: int64

In [23]:
points_joined["redlistCategory"] = points_joined["redlistCategory"].replace({"Critically Endangered":"CR",
                                                                             "Endangered":"EN",
                                                                             "Near Threatened": "NT",
                                                                             "Vulnerable": "VU"
                                                                             })

In [24]:
points_joined = points_joined.drop(columns=["redlistCriteria"])


In [25]:
points_joined.head()

,sci_name,presence,origin,seasonal,lon,lat,redlistCategory
0,Hubbsina turneri,1,1,1,-101.47950,19.87450,CR
1,Hubbsina turneri,1,1,1,-101.77730,19.82560,CR
2,Hubbsina turneri,1,1,1,-101.78760,19.82730,CR
3,Ictalurus mexicanus,1,1,1,-99.35417,21.98083,VU
4,Ictalurus mexicanus,1,1,1,-99.30000,22.00000,VU


In [36]:
points_joined.to_csv("../data/GeoDataFrame/dataframe/df_sp.csv", index = False)

In [27]:
points_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70273 entries, 0 to 70272
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sci_name         70273 non-null  object 
 1   presence         70273 non-null  int64  
 2   origin           70273 non-null  int64  
 3   seasonal         70273 non-null  int64  
 4   lon              70273 non-null  float64
 5   lat              70273 non-null  float64
 6   redlistCategory  70273 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 3.8+ MB


In [29]:
points_joined["redlistCategory"].unique()

array(['CR', 'VU', 'EN', 'NT'], dtype=object)